# OpenEO Connection to EODC Backend

In [1]:
import logging
import openeo
from openeo.auth.auth_bearer import BearerAuth
logging.basicConfig(level=logging.INFO)

In [2]:
# Define constants

# Connection
EODC_DRIVER_URL = "http://openeo.eodc.eu"
OUTPUT_FILE = "/tmp/openeo_eodc_output.tiff"

EODC_USER = "user1"
EODC_PWD = "Test123#"

# Data
PRODUCT_ID = "s2a_prd_msil1c"

DATE_START = "2017-01-01"
DATE_END = "2017-01-08"

IMAGE_LEFT = 652000
IMAGE_RIGHT = 672000
IMAGE_TOP = 5161000
IMAGE_BOTTOM = 5181000
IMAGE_SRS = "EPSG:32632"

# Processes
NDVI_RED = "B04"
NDVI_NIR = "B08"

In [3]:
# Connect with EODC backend
session = openeo.session(EODC_USER, endpoint=EODC_DRIVER_URL)
# Login 
token = session.auth(EODC_USER, EODC_PWD, BearerAuth)

session

TypeError: get_base() missing 1 required positional argument: 'self'

In [4]:
# Retrieve the list of available collections
collections = session.imagecollections()

list(collections)[:5]

[{'description': 'Sentinel-2 is a multispectral, high-resolution, optical imaging mission, developed by the European Space Agency (ESA) in the frame of the Copernicus program of the European Commission.',
  'product_id': 's2a_prd_msil1c',
  'source': 'ESA'}]

In [5]:
# Get detailed information about a collection
collection = session.get_collection(PRODUCT_ID)

collection

{'dc:creator': 'ESA',
 'dc:date': '2017-12-05',
 'dc:identifier': 's2a_prd_msil1c',
 'dc:language': 'eng',
 'dc:relation': None,
 'dc:rights': 'otherRestrictions',
 'dc:subject': ['Sentinel-2',
  'Orthoimagery',
  'Land cover',
  'Geographical names',
  'ISO19115',
  'dataset',
  'imageryBaseMapsEarthCover',
  'no conditions apply',
  'Remote Sensing',
  'http://www.opengis.net/def/crs/EPSG/0/4936',
  'otherRestrictions',
  'EODC Earth Observation Data Centre GmbH',
  'no limitation',
  'Satellite',
  'S2MSIL1C',
  'Level-1C',
  'Copernicus',
  'Sentinel-2 MSI Level-1C',
  {'#text': 'imageryBaseMapsEarthCover',
   '@scheme': 'http://www.isotc211.org/2005/resources/Codelist/gmxCodelists.xml#MD_TopicCategoryCode'}],
 'dc:title': 'Sentinel-2A MSI Level-1C',
 'dc:type': 'series',
 'dct:abstract': 'Sentinel-2 is a multispectral, high-resolution, optical imaging mission, developed by the European Space Agency (ESA) in the frame of the Copernicus program of the European Commission.',
 'dct:al

In [6]:
# Select collection product
s2a_prd_msil1c = session.image(PRODUCT_ID)

s2a_prd_msil1c

In [7]:
# Get list of available processes
processes = session.get_all_processes()

processes

[{'description': 'Converts and outputs the data to output destination.',
  'process_id': 'convert'},
 {'description': 'Allows to extract one or multiple bands of multi-band raster image collection. Bands can be chosen either by band id, band name or by wavelength.',
  'process_id': 'filter_bands'},
 {'description': 'Drops observations from a collection that are located outside of a given bounding box.',
  'process_id': 'filter_bbox'},
 {'description': 'Drops observations from a collection that have been captured before a start or after a given end date.',
  'process_id': 'filter_daterange'},
 {'description': 'Extracts and filters Sentinel-2 data from EODC storage to a mounted volume.',
  'process_id': 'filter-s2'},
 {'description': 'Finds the minimum value of time series for all bands of the input dataset.',
  'process_id': 'min_time'},
 {'description': 'Finds the minimum value of time series for all bands of the input dataset.',
  'process_id': 'NDVI'}]

In [8]:
# Specifying the date range and the bounding box
timeseries = s2a_prd_msil1c.filter_bbox(west=IMAGE_LEFT, east=IMAGE_RIGHT, north=IMAGE_TOP,
                                             south=IMAGE_BOTTOM, crs=IMAGE_SRS)
timeseries = timeseries.filter_temporal(DATE_START, DATE_END)

timeseries

In [9]:
# Applying some operations on the data
timeseries = timeseries.ndvi(NDVI_RED, NDVI_NIR)
timeseries = timeseries.min_time()

timeseries

In [10]:
# Sending the job to the backend
job = timeseries.create_job()

job

In [11]:
# Queue job to start the processing
status = job.queue()

In [11]:
# Get Status of Job
status = job.status()

status

{'consumed_credits': 0.0,
 'job_id': 109,
 'status': 'submitted',
 'submitted': 'Tue, 13 Mar 2018 17:57:43 GMT',
 'task': {'process_graph': {'args': {'imagery': {'args': {'imagery': {'args': {'from': '2017-01-01',
        'imagery': {'args': {'bottom': 5181000,
          'imagery': {'product_id': 's2a_prd_msil1c'},
          'left': 652000,
          'right': 672000,
          'srs': 'EPSG:32632',
          'top': 5161000},
         'process_id': 'filter_bbox'},
        'to': '2017-01-08'},
       'process_id': 'filter_daterange'},
      'nir': 'B08',
      'red': 'B04'},
     'process_id': 'NDVI'}},
   'process_id': 'min_time'}},
 'updated': 'Tue, 13 Mar 2018 17:57:43 GMT',
 'user_id': 2}

In [12]:
# Download job result
from openeo.rest.job import ClientJob
job = ClientJob(107, session)
job.download(OUTPUT_FILE)

job

Files are not (yet) available for job with id 107.


In [2]:
# Showing the result
!gdalinfo -hist "/tmp/openeo_eodc_output.tiff"

Driver: GTiff/GeoTIFF
Files: /tmp/openeo_eodc_output.tiff
       /tmp/openeo_eodc_output.tiff.aux.xml
Size is 953, 622
Coordinate System is:
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0],
    UNIT["degree",0.0174532925199433],
    AUTHORITY["EPSG","4326"]]
Origin = (10.984051868786896,46.760379340476071)
Pixel Size = (0.000281691226781,-0.000281691226781)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (  10.9840519,  46.7603793) ( 10d59' 2.59"E, 46d45'37.37"N)
Lower Left  (  10.9840519,  46.5851674) ( 10d59' 2.59"E, 46d35' 6.60"N)
Upper Right (  11.2525036,  46.7603793) ( 11d15' 9.01"E, 46d45'37.37"N)
Lower Right (  11.2525036,  46.5851674) ( 11d15' 9.01"E, 46d35' 6.60"N)
Center      (  11.1182777,  46.6727734) ( 11d 7' 5.80"E, 46d40'21.98"N)
Band 1 Block=953x2 Type=Float32, ColorInterp=